# Multiple Linear Regression with Sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
sns.set()

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
data = pd.read_csv('1.02. Multiple linear regression.csv')

In [5]:
data.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


In [6]:
# sample is the machine learning word for observation

data.describe()

,SAT,GPA,"Rand 1,2,3"
count,84.000000,84.000000,84.000000
mean,1845.273810,3.330238,2.059524
std,104.530661,0.271617,0.855192
min,1634.000000,2.400000,1.000000
25%,1772.000000,3.190000,1.000000
50%,1846.000000,3.380000,2.000000
75%,1934.000000,3.502500,3.000000
max,2050.000000,3.810000,3.000000


In [7]:
x = data[['SAT','Rand 1,2,3']]

In [8]:
y = data['GPA']

In [9]:
reg = LinearRegression()

In [10]:
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
# it is the coefficient of the SAT and the Rand 1,2,3
# they are ordered in the way we fed them
# the 0.0016 corresponds to SAT
# the -0.0083 corresponds to Rand 1,2,3

reg.coef_

array([ 0.00165354, -0.00826982])

In [12]:
reg.intercept_

0.29603261264909486

In [13]:
# the R-squared is a universal measure to evaluate how well linear regression fare and compare

reg.score(x,y)

0.4066811952814285

## The Adjusted R-squared

The adjusted R-squared is much more appropriate measure for a multiple linear regression.

The adjusted R-squared steps on the R-squared and adjusts for the number of variables included in the model.

There is no adjusted R-squared methods in the package.

The Adjusted R-squared formule is:

$R^2{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In this formula, n = 84, it means the number of observations; the p = 2, it means the number of predictors.

In [14]:
x.shape

(84, 2)

In [15]:
r2 = reg.score(x,y)

n = x.shape[0]
p = x.shape[1]

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [16]:
# the adjusted R-squared is lower than R-squared
# therefore one or more of the predictors have littel or no explanatory power

adjusted_r2

0.39203134825134023

## Feature Selection

Through the adjusted R-squared, it can one of the two variables doesn't increase the explanatory power of the regression, therefore if we can identify which one and remove it we would further simplify this model.
___

How to detect the variables which are unneded in a model ?

There's actually a whole process created for this purpose, it is called feature selection, feature selection is a very important procedure in machine learning as it simplifies models which makes them much easier to interpret by data scientists.

Feature selection simplifies models, improve speed and often prevent a series of unwanted issues arising from having too many features.
___

Through the p-value can determine whether the independent variables were relevant for the model, if a p-value > 0.05, we can disregard it.

How can we find the p-value in sklearn, the sklearn as a machine learning package rather than statistics one, so there is no built in method we can call to get p-value.

Therefore we must find a work around a very close concept is available with the feature selection module from sklearn, it is called f-regression, f-regression creates simple linear regressions of each feature and the depedent variable, for this GPA and SAT example, it would translate into two regressions, one where we predict GPA with SAT and the other where we predict GPA with Rand 1,2,3.

Then the method would calculate the F-statistic for each of those regressions and return the respective p-values.

If there were 50 features, 50 simple regression would be created.

For a simple regression the p-value of the F-statistic coincides with the p-value of the only independent variable, therefore this method is precisely what we need.

In [17]:
from sklearn.feature_selection import f_regression

In [18]:
# the output are all arrays
# the first one contains the F-statistics for each of the regressions
# the second one contains the p-value
# generally we are interested only in the p-value

f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [19]:
p_values = f_regression(x,y)[1]

In [20]:
# it is scientific notation
# e-11 means *10^-11 or /10^11
# e-1 means *10^-1 or /10^1

p_values

array([7.19951844e-11, 6.76291372e-01])

In [21]:
# the first p-value refers to the first column of SAT
# the second p-value refers to the second column of Rand 1,2,3
# so the p-value of SAT is 0.000
# the p-value of Rand 1,2,3 is 0.676
# in this case the SAT is useful variable while Rand 1,2,3 is useless
# there are the univariate p-values reached from simple linear models
# they don't reflect the interconnection of the features in our multiple linear regression
# there f-regression should be used with caution

p_values.round(3)

array([0.   , 0.676])

## Summary Table 

In [22]:
reg_summary = pd.DataFrame(data = x.columns.values, columns = ['Features'])

In [23]:
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [24]:
reg_summary['Coefficients'] = reg.coef_
reg_summary['p-value'] = p_values.round(3)

In [25]:
# through the p-value we can know the Rand 1,2,3 should be removed
# p-values are one of the best ways to determine if a variable is redundant
# but they provide no information whatsoever about how useful a variable is
# maybe both p-value are all 0.000 but this doesn't mean they are equally important

reg_summary

,Features,Coefficients,p-value
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


## Feature Scaling

The most common problem in working with numerical data is the difference in magnitudes.

An easy fix for this issue is standardization also know as feature scaling or normalization.

However normalization could also refet to a few additional concepts within machine learning.

Standardization or feature scaling is the process of transforming data into a standard scale.

This translates to subtracting the mean and dividing by the standard deviation.

In this way, we will obtain a distribution with a mean of 0 and standard deviation of 1 which could easily be proven.

Through the standarization, it can forced of very different scales to appear similar.

Use the feature scaling can ensure linear regressions treat the two variables equally and it is much easier to make sense of the data.

There are different strategies to deal with different magnitudes, however standardization is probably the most common one.

In [26]:
x = data[['SAT','Rand 1,2,3']]

In [27]:
y = data['GPA']

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
# scaler be used to sclae our data

scaler = StandardScaler()

In [30]:
# fit can calculates and stores the mean and standard deviation of each feature

scaler.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [31]:
x_scaler = scaler.transform(x)

In [32]:
# using the whole steps the whole data has been standardized
# having all inputs with the same magnitude allows us to compart their impact

x_scaler

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

In [33]:
reg = LinearRegression()

In [34]:
reg.fit(x_scaler,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [36]:
reg.intercept_

3.330238095238095

In [37]:
# Weights will include the intercept and two coefficients

reg_summary = pd.DataFrame([['Intercept'],['SAT'],['Rand 1,2,3']], columns = ['Features'])
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

In [38]:
# Weights is a the machine learning word for coefficients
# we have standardized the features which resulted in standardized coefficients or weights
# the bigger the weight, the bigger the impact of the feature of the regression
# the machine learning word for intercept is bias
# the intercept is nothing but a number which adjusted our regression with some constant
# if we need to adjust our regression with some constant then the regression is biased by that number

reg_summary

,Features,Weights
0,Intercept,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


In [39]:
reg_summary = pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']], columns = ['Features'])
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

In [40]:
# the close a weight is to 0, the smaller its impact
# the bigger the weight, the bigger its impact
# the weight of SAT is much much bigger than Rand 1,2,3
# this brings us to feature selection through standardization
# we can clearly see that Rand 1,2,3 barely contributes to our output if at all

reg_summary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


We can clearly see that Rand 1,2,3 barely contributes to our output if at all, it will make little difference if we remove it from the model or leave it there with a weight of almost 0.

This concept is quite remarkable, when we perform feature scaling, we don't really care if a useless variable is there or not.

This is also one of the main reasons why sklearn doesn't natively support p-value since most perform some kine of feature scaling before fitting a model, we don't really need to identify the worst performing features, they are automatically penalized by having a very small weight.

In general it always prefer to leave out the worst perfoming features as they interact with the useful ones and may bias the weights even if only slightly so.

## Predict values using standardized model

In [41]:
new_data = pd.DataFrame(data = [[1700,2], [1800,1]], columns = ['SAT','Rand 1,2,3'])

In [42]:
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [43]:
# this is not the GPA
# the reason is the regression model now is trained on standardized inputs
# it expected values that are of the same magnitude as the ones used in the trained process
# so the new data should be arranged in the same way and also must be standardized in the same way
# in the same mean and standard deviation

reg.predict(new_data)

array([295.39979563, 312.58821497])

In [44]:
new_data_scaled = scaler.transform(new_data)

In [45]:
# with the scaler.transform the new_data
# this data in the array which contains are standardized data

new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [46]:
# so it is the right predict

reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

## What if removed the Rand 1,2,3

In [47]:
reg_simple = LinearRegression()

In [48]:
# only the SAT column

x_simple_matrix = x_scaler[:,0].reshape(-1,1)

In [49]:
reg_simple.fit(x_simple_matrix,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [50]:
# the same it is only the SAT column
# it get the same results all the same 3.09 and 3.26
# when we apply the feature scaling it often doesn't affect the final result
# if we keep or leave out in significant features
# the weights will be so close to zero that they will barely influence the predictions

reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 3.25527879])